In [1]:
import pandas as pd
import re
from sklearn.decomposition import IncrementalPCA

In [2]:
# Load Foldseek results
df = pd.read_csv(
    "../data/foldseek_allvsall.tsv",
    sep="\t",
    names=["query", "target", "evalue", "bitscore", "tmscore"],
)

In [3]:
print(df.shape)
df.head()

(43597570, 5)


,query,target,evalue,bitscore,tmscore
0,OP172784.1_prot_WAX12964.1_210,OP172784.1_prot_WAX12964.1_210,2.929000e-08,349,1.0160
1,OP172784.1_prot_WAX12964.1_210,OR999189.1_prot_WYD66467.1_165,1.310000e-07,324,1.0110
2,OP172784.1_prot_WAX12964.1_210,MH553563.1_prot_AXF42419.1_111,1.310000e-07,323,1.0100
3,OP172784.1_prot_WAX12964.1_210,NC_054906.1_prot_YP_010066992.1_111,8.121000e-07,294,0.9971
4,OP172784.1_prot_WAX12964.1_210,MK524178.1_prot_QBP05723.1_196,1.558000e-06,271,0.9808


In [4]:
# Add column for phage ids
pattern = re.compile(r"^(?P<accession>.+?)_prot_")

df["Accession"] = df["query"].apply(lambda x: pattern.match(x).group("accession"))

# Drop accession version (e.g. xxx.1)
df["Accession"] = df["Accession"].apply(
    lambda x: x.split(".")[0] if isinstance(x, str) else x
)

In [5]:
# Keep relevant cols
df = df[["Accession", "query", "target", "evalue"]]
df

,Accession,query,target,evalue
0,OP172784,OP172784.1_prot_WAX12964.1_210,OP172784.1_prot_WAX12964.1_210,2.929000e-08
1,OP172784,OP172784.1_prot_WAX12964.1_210,OR999189.1_prot_WYD66467.1_165,1.310000e-07
2,OP172784,OP172784.1_prot_WAX12964.1_210,MH553563.1_prot_AXF42419.1_111,1.310000e-07
3,OP172784,OP172784.1_prot_WAX12964.1_210,NC_054906.1_prot_YP_010066992.1_111,8.121000e-07
4,OP172784,OP172784.1_prot_WAX12964.1_210,MK524178.1_prot_QBP05723.1_196,1.558000e-06
...,...,...,...,...
43597565,KP671755,KP671755.1_prot_AJT61184.1_343,OR352944.1_prot_WPK34943.1_41,8.122000e+00
43597566,KP671755,KP671755.1_prot_AJT61184.1_343,OQ828463.1_prot_WPJ68961.1_1,8.677000e+00
43597567,KP671755,KP671755.1_prot_AJT61184.1_343,MZ751040.1_prot_UAV84284.1_14,8.677000e+00
43597568,KP671755,KP671755.1_prot_AJT61184.1_343,NC_005135.1_prot_NP_932501.1_146,9.902000e+00


In [6]:
# For each phage-protein pair, keep the lowest evalue
phage_target_min = df.groupby(["Accession", "target"])["evalue"].min().reset_index()

# rotate into phage x protein matrix
matrix = phage_target_min.pivot(index="Accession", columns="target", values="evalue")

# Replace missing with 10 (arbitrary high)
matrix = matrix.fillna(10)
matrix

target,AB334721.1_prot_BAF74213.1_2,AB334721.1_prot_BAF74214.1_3,AB334721.1_prot_BAF74216.1_5,AB334721.1_prot_BAF74217.1_6,AB334721.1_prot_BAF74218.1_7,AB334721.1_prot_BAF74219.1_8,AB334721.1_prot_BAF74220.1_9,AB334721.1_prot_BAF74221.1_10,AB334721.1_prot_BAF74222.1_1,AB903967.1_prot_BAO47048.1_1,...,PV786929.1_prot_XYW65624.1_16,PV786929.1_prot_XYW65625.1_17,X60323.1_prot_CAA42884.1_1,X60323.1_prot_CAA42885.1_2,X60323.1_prot_CAA42886.1_4,X60323.1_prot_CAA42887.1_3,X60323.1_prot_CAA42888.1_5,X60323.1_prot_CAA42891.1_8,X60323.1_prot_CAA42892.1_9,X60323.1_prot_CAA42893.1_10
Accession,,,,,,,,,,,,,,,,,,,,,
AB334721,1.756000e-22,7.473000e-15,0.00074,1.401000e-10,3.980000e-75,1.528000e-15,4.845000e-76,2.199000e-81,6.777000e-89,1.000000e+01,...,1.000000e+01,1.704000e+00,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.175000e-02,1.000000e+01
AB903967,5.971000e+00,1.372000e-01,10.00000,1.000000e+01,4.051000e+00,1.000000e+01,1.000000e+01,5.548000e+00,1.000000e+01,2.121000e-20,...,1.000000e+01,3.268000e-15,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,7.169000e-04,1.000000e+01
AB983711,1.000000e+01,1.000000e+01,10.00000,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,...,1.000000e+01,1.857000e-17,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.394000e+00,1.000000e+01
AF063097,1.000000e+01,1.000000e+01,10.00000,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,...,1.000000e+01,1.000000e+01,3.250000e-03,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,2.919000e+00,1.000000e+01
AF115103,1.000000e+01,8.520000e+00,10.00000,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,...,1.000000e+01,1.788000e+00,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,7.368000e+00,1.000000e+01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PV660458,1.000000e+01,1.000000e+01,10.00000,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,8.800000e+00,1.000000e+01,1.000000e+01,...,1.000000e+01,1.095000e-19,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,8.800000e+00,1.000000e+01
PV683289,1.000000e+01,4.074000e+00,10.00000,1.000000e+01,8.095000e+00,1.000000e+01,3.852000e-01,5.038000e+00,1.000000e+01,2.513000e+00,...,4.712000e+00,6.156000e-02,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,5.913000e-02,4.372000e+00
PV775412,1.990000e+00,6.538000e-02,10.00000,1.000000e+01,6.185000e+00,1.000000e+01,3.315000e-03,1.000000e+01,1.000000e+01,2.471000e-14,...,1.000000e+01,3.516000e-02,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,2.299000e-01,1.000000e+01


In [7]:
# Perform PCA
pca = IncrementalPCA(n_components=2)  # 2D or 3D
coords = pca.fit_transform(matrix)

print(f"Explained variance: {pca.explained_variance_ratio_}")

Explained variance: [0.28096382 0.05020559]


In [8]:
# Import phage matadata
phage_meta = pd.read_csv("../data/ncbi_phage_metadata.tsv", sep="\t", low_memory=False)

# Combine PCA coordinates with family info
coords_df = pd.DataFrame(coords, index=matrix.index, columns=["PC1", "PC2"])
coords_df = coords_df.reset_index().rename(columns={"index": "Accession"})

# Merge with metadata by accession
coords_annot = coords_df.merge(
    phage_meta[["Accession", "Family", "Species"]],
    on="Accession",
    how="left",  # keep all phages that were in the PCA
)

# Store annotated coordinates
coords_annot.to_csv("../data/phage_PCA_coords_annot.tsv", sep="\t", index=False)